# Bag of Words model for anime titles
bag of words model for titles of anime.
also should probably do it for genres too
going to be looking at the overall animelist on MAL (not user animelists) for anime titles and genres

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# load dataset
animelist_data = pd.read_csv('anime_list_cleaned.csv')

In [3]:
animelist_data.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0


In [6]:
# vocabulary

IGNORE = {
    'of', 'the', 'and', '-', 'in', 'a', '&', 'my', 'to', 'i', 'ii', '2', '3', '4', 'is', 'on', 'no', 'x'
}

titles = animelist_data[animelist_data['title_english'].notnull()]['title_english']
begin_list = ' '.join(list(titles)).replace('"', '').replace(':', '').lower()

ignore_done_list = begin_list.split(' ')
ignore_done_list = [i for i in ignore_done_list if i not in IGNORE]

s = pd.Series((ignore_done_list))
s.groupby(s).value_counts().sort_values(ascending=False).iloc[0:50]



movie      movie        135
season     season        64
girl       girl          60
dragon     dragon        52
love       love          51
school     school        47
girls      girls         46
pokemon    pokemon       46
black      black         40
adventure  adventure     39
princess   princess      39
legend     legend        37
ova        ova           35
gundam     gundam        35
high       high          35
ghost      ghost         35
mobile     mobile        35
special    special       35
ball       ball          34
world      world         33
specials   specials      32
blue       blue          31
animation  animation     31
magical    magical       31
samurai    samurai       31
star       star          31
z          z             29
suit       suit          28
time       time          27
moon       moon          26
little     little        26
you        you           25
new        new           25
second     second        24
life       life          24
chapter    chapter  

In [12]:
# get the top 50 and make a many-hot out of them
top_50 = s.groupby(s).value_counts().sort_values(ascending=False).head(50)

top_50 = [i[0] for i in list(top_50.index)]

In [18]:
len(top_50)

50

In [15]:
ald_copy = animelist_data.copy()

ald_copy.drop(ald_copy.columns.difference(['anime_id', 'title_english']), 1, inplace=True)

In [22]:
ald_copy

,anime_id,title_english
0,11013,Inu X Boku Secret Service
1,2104,My Bride is a Mermaid
2,5262,Shugo Chara!! Doki
3,721,Princess Tutu
4,12365,Bakuman.
...,...,...
6663,37405,NaN
6664,37886,I&#039;m Joybo OVA
6665,37255,NaN
6666,35229,Mr. Men Little Miss


In [27]:
manyhots_added = pd.concat([ald_copy, pd.DataFrame(0, index=np.arange(len(ald_copy)), columns=top_50)], axis=1)

In [28]:
manyhots_added

,anime_id,title_english,movie,season,girl,dragon,love,school,girls,pokemon,...,battle,one,night,hero,from,space,story,cat,war,prince
0,11013,Inu X Boku Secret Service,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2104,My Bride is a Mermaid,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5262,Shugo Chara!! Doki,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,721,Princess Tutu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12365,Bakuman.,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,37405,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6664,37886,I&#039;m Joybo OVA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6665,37255,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6666,35229,Mr. Men Little Miss,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
mha_copy = manyhots_added.copy()
for i, row in manyhots_added.iterrows():
    title = row[1]
    if type(title) is not float: 
        for token in top_50:
            count = title.lower().split(' ').count(token)
            mha_copy.at[i, token] = count

In [50]:
mha_copy.sum(axis = 0, skipna = True)

anime_id     100453272
movie               98
season              64
girl                53
dragon              51
love                50
school              47
girls               43
pokemon             20
black               38
adventure           34
princess            36
legend              37
ova                 34
gundam              27
high                35
ghost               35
mobile              35
special             33
ball                32
world               33
specials            32
blue                31
animation           31
magical             31
samurai             30
star                28
z                   26
suit                28
time                26
moon                20
little              26
you                 25
new                 25
second              24
life                22
chapter             23
blade               18
with                23
digimon             18
super               22
battle              22
one                 22
night      